# Wrangle Exercises

In [1]:
import pyspark
from pyspark.sql.functions import *

spark = pyspark.sql.SparkSession.builder.getOrCreate()

## Data Prep

In [13]:
df = case = spark.read.csv('data/case.csv', header=True, inferSchema=True)
print('nrows:', df.count())
stray_animal_cases = df.filter(df.service_request_type == 'Stray Animal').count()
print('stray animal cases:', stray_animal_cases)

nrows: 841704
stray animal cases: 26760


In [26]:
# Rename column
df = df.withColumnRenamed('SLA_due_date', 'case_due_date')

# Convert to better data types
df = (
    df.withColumn('case_late', col('case_late') == 'YES')
    .withColumn('case_closed', col('case_closed') == 'YES')
)
df = df.withColumn('council_district', format_string('%04d', col('council_district')))
df = (
    df.withColumn('case_opened_date', to_timestamp(col('case_opened_date'), 'M/d/yy H:mm'))
    .withColumn('case_closed_date', to_timestamp(col('case_closed_date'), 'M/d/yy H:mm'))
    .withColumn('case_due_date', to_timestamp(col('case_due_date'), 'M/d/yy H:mm'))
)

# Cleanup text data
df = df.withColumn('request_address', lower(trim(col('request_address'))))
# Extract zipcode
df = df.withColumn('zipcode', regexp_extract(col('request_address'), r'\d+$', 0))

# Create a `case_lifetime` feature
df = (
    df.withColumn('case_age', datediff(current_timestamp(), 'case_opened_date'))
    .withColumn('days_to_closed', datediff('case_closed_date', 'case_opened_date'))
    .withColumn('case_lifetime', when(col('case_closed'), col('days_to_closed')).otherwise(col('case_age')))
    .drop('case_age', 'days_to_closed')
)

# Join departments and sources
depts = spark.read.csv('data/dept.csv', header=True, inferSchema=True)
sources = spark.read.csv('data/source.csv', header=True, inferSchema=True)

df = df.join(depts, 'dept_division', 'left').join(sources, 'source_id', 'left')

# # Train Test Split
# train, test = df.randomSplit([.8, .2], seed=123)
# train, validate, test = df.randomSplit([.7, .15, .15], seed=123)

In [4]:
df.count()

855269

In case we want to query our dataframe with Spark SQL:

In [5]:
df.createOrReplaceTempView('df')

## Initial Exploration Questions

In [6]:
# How old is the latest (in terms of days past SLA) currently open issue?
# How long has the oldest (in terms of days since opened) currently opened issue been open?
spark.sql('''
SELECT DATEDIFF(current_timestamp, case_due_date) AS days_past_due
FROM df
WHERE NOT case_closed
ORDER BY days_past_due DESC
LIMIT 15
''').show()

+-------------+
|days_past_due|
+-------------+
|         1414|
|         1414|
|         1414|
|         1413|
|         1411|
|         1407|
|         1407|
|         1406|
|         1405|
|         1405|
|         1401|
|         1400|
|         1400|
|         1400|
|         1397|
+-------------+



In [7]:
# How many Stray Animal cases are there?
df.filter(df.service_request_type == 'Stray Animal').count()

27361

26760?

In [8]:
(
    df.groupBy('service_request_type')
    .count()
    .filter(expr('service_request_type == "Stray Animal"'))
    .show()
)

+--------------------+-----+
|service_request_type|count|
+--------------------+-----+
|        Stray Animal|27361|
+--------------------+-----+



In [9]:
# How many service requests that are assigned to the Field Operations department (dept_division)
# are not classified as "Officer Standby" request type (service_request_type)?
(
    df.filter(df.dept_division == 'Field Operations')
    .filter(df.service_request_type != 'Officer Standby')
    .count()
)

116295

In [10]:
# Another way to do it
(
    df.filter(expr("dept_division == 'Field Operations'"))
    .filter(expr('service_request_type != "Officer Standby"'))
    .count()
)

116295

In [ ]:
# Convert the council_district column to a string column.

# Already done in the data prep

In [11]:
# Extract the year from the case_closed_date column.
df.select('case_closed_date', year('case_closed_date')).show(5)

+-------------------+----------------------+
|   case_closed_date|year(case_closed_date)|
+-------------------+----------------------+
|2018-01-01 12:29:00|                  2018|
|2018-01-03 08:11:00|                  2018|
|2018-01-02 07:57:00|                  2018|
|2018-01-02 08:13:00|                  2018|
|2018-01-01 13:29:00|                  2018|
+-------------------+----------------------+
only showing top 5 rows



In [12]:
# Convert num_days_late from days to hours in new columns num_hours_late.
(
    df.withColumn('num_hours_late', df.num_days_late * 24)
    .select('num_days_late', 'num_hours_late')
    .show()
)

+-------------------+-------------------+
|      num_days_late|     num_hours_late|
+-------------------+-------------------+
| -998.5087616000001|     -23964.2102784|
|-2.0126041669999997|-48.302500007999996|
|       -3.022337963|      -72.536111112|
|       -15.01148148|      -360.27555552|
|0.37216435200000003|  8.931944448000001|
|       -29.74398148| -713.8555555199999|
|       -14.70673611|      -352.96166664|
|       -14.70662037|      -352.95888888|
|       -14.70662037|      -352.95888888|
|       -14.70649306|      -352.95583344|
|       -14.70649306|      -352.95583344|
|       -14.70636574|      -352.95277776|
|          -14.70625|-352.95000000000005|
|       -14.70636574|      -352.95277776|
|       -14.70623843|-352.94972232000003|
|-14.705891199999998|-352.94138879999997|
|       -14.70600694|      -352.94416656|
|       -14.70576389|      -352.93833336|
|       -14.70576389|      -352.93833336|
|       -14.70564815|       -352.9355556|
+-------------------+-------------

In [ ]:
# Join the case data with the source and department data.

# already joined in the data prep

In [16]:
sources.show()

+---------+--------------------+
|source_id|     source_username|
+---------+--------------------+
|   100137|    Merlene Blodgett|
|   103582|         Carmen Cura|
|   106463|     Richard Sanchez|
|   119403|      Betty De Hoyos|
|   119555|      Socorro Quiara|
|   119868| Michelle San Miguel|
|   120752|      Eva T. Kleiber|
|   124405|           Lori Lara|
|   132408|       Leonard Silva|
|   135723|        Amy Cardenas|
|   136202|    Michelle Urrutia|
|   136979|      Leticia Garcia|
|   137943|    Pamela K. Baccus|
|   138605|        Marisa Ozuna|
|   138650|      Kimberly Green|
|   138650|Kimberly Green-Woods|
|   138793| Guadalupe Rodriguez|
|   138810|       Tawona Martin|
|   139342|     Jessica Mendoza|
|   139344|        Isis Mendoza|
+---------+--------------------+
only showing top 20 rows



In [18]:
sources.sort(col('source_id').desc()).show()

+---------+-----------------+
|source_id|  source_username|
+---------+-----------------+
|  yh24110| Yojani Hernandez|
|  yc16753|    Yvonne Casias|
|  vk26526|    Vincent Kosub|
|  vb22265| Vanessa Burciaga|
|  ts15690|  Thelma Martinez|
|  sw26367|Samantha Wickwire|
| svcCRMSS|         svcCRMSS|
| svcCRMLS|         svcCRMLS|
| svcCFlag|         CityFlag|
|  sv24848|  Salvador Valdez|
|  ss26317|     Sakib Shaikh|
|  ss21394|  Shirley Sanchez|
|  ss09159|   Sylvia Sanchez|
|  sp26368|  Steven Peterson|
|  sg26196|   Samantha Garza|
|  sg22264|  Steven Martinez|
|  ru26699|      Raul Ubides|
|  rs16746|Ronald S. Sorrell|
|  rb05270|       Rene Bomar|
|  ps01944|   Paula Stallcup|
+---------+-----------------+
only showing top 20 rows



In [21]:
(
    case.join(sources, "source_id", "left")
    .sort(col("source_username"))
    .show(5, vertical=True)
)

-RECORD 0------------------------------------
 source_id            | af26445              
 case_id              | 1014220226           
 case_opened_date     | 2/5/18 15:07         
 case_closed_date     | 2/5/18 17:59         
 SLA_due_date         | 6/13/18 15:07        
 case_late            | NO                   
 num_days_late        | -127.88046299999999  
 case_closed          | YES                  
 dept_division        | Streets              
 service_request_type | Base/Pavement Repair 
 SLA_days             | 128.0                
 case_status          | Closed               
 request_address      | BAYWATER DR and L... 
 council_district     | 7                    
 source_username      | Alex Franklin        
-RECORD 1------------------------------------
 source_id            | af26445              
 case_id              | 1014218339           
 case_opened_date     | 2/5/18 9:17          
 case_closed_date     | 2/7/18 15:57         
 SLA_due_date         | 2/19/18 9:

In [22]:
# Are there any cases that do not have a request source?
# are there any null values for source_id?
(
    df.select(df.source_id.isNull().cast('int').alias('is_null'))
    .agg(sum('is_null'))
    .show()
)

+------------+
|sum(is_null)|
+------------+
|           0|
+------------+



In [23]:
df.filter(col('source_id').isNull()).show(vertical=True)

(0 rows)



In [24]:
# What are the top 10 service request types in terms of number of requests?
(
    df.groupby('service_request_type')
    .count()
    .sort(col('count').desc())
    .show(10, truncate=False)
)

+--------------------------------+-----+
|service_request_type            |count|
+--------------------------------+-----+
|No Pickup                       |86855|
|Overgrown Yard/Trash            |65895|
|Bandit Signs                    |32910|
|Damaged Cart                    |30338|
|Front Or Side Yard Parking      |28794|
|Stray Animal                    |26760|
|Aggressive Animal(Non-Critical) |24882|
|Cart Exchange Request           |22024|
|Junk Vehicle On Private Property|21473|
|Pot Hole Repair                 |20616|
+--------------------------------+-----+
only showing top 10 rows



In [27]:
# What are the top 10 service request types in terms of average days late?
# - just the late cases
# - for the late cases:
#   - what is the average number of days late by request type?
(
    df.where('case_late') # just the rows where case_late == true
    .groupBy('service_request_type')
    .agg(mean('num_days_late').alias('n_days_late'), count('*').alias('n_cases'))
    .sort(desc('n_days_late'))
    .show(10, truncate=False)
)

+--------------------------------------+------------------+-------+
|service_request_type                  |n_days_late       |n_cases|
+--------------------------------------+------------------+-------+
|Zoning: Recycle Yard                  |210.89201994318182|132    |
|Zoning: Junk Yards                    |200.20517608494276|262    |
|Structure/Housing Maintenance         |190.20707698509807|51     |
|Donation Container Enforcement        |171.09115313942615|122    |
|Storage of Used Mattress              |163.96812829714287|7      |
|Labeling for Used Mattress            |162.43032902285717|7      |
|Record Keeping of Used Mattresses     |153.99724039428568|7      |
|Signage Requied for Sale of Used Mattr|151.63868055333333|12     |
|Traffic Signal Graffiti               |137.64583330000002|4      |
|License Requied Used Mattress Sales   |128.79828704142858|7      |
+--------------------------------------+------------------+-------+
only showing top 10 rows



In [28]:
# Does number of days late depend on department?
(
    df.filter('case_late')
    .groupby('dept_name')
    .agg(mean('num_days_late').alias('days_late'), count('num_days_late').alias('n_cases_late'))
    .sort('days_late')
    .withColumn('days_late', round(col('days_late'), 1))
    .show(truncate=False)
)

+-------------------------+---------+------------+
|dept_name                |days_late|n_cases_late|
+-------------------------+---------+------------+
|Metro Health             |6.5      |854         |
|Solid Waste Management   |7.1      |33729       |
|Trans & Cap Improvements |10.7     |5529        |
|Parks and Recreation     |22.4     |3810        |
|Animal Care Services     |23.4     |23751       |
|Code Enforcement Services|48.1     |25467       |
|Development Services     |67.2     |840         |
|Customer Service         |88.2     |2035        |
|null                     |210.9    |132         |
+-------------------------+---------+------------+



In [29]:
df.groupby('dept_name').count().show(truncate=False)

+-------------------------+------+
|dept_name                |count |
+-------------------------+------+
|Animal Care Services     |119362|
|null                     |198   |
|Solid Waste Management   |286287|
|Development Services     |1397  |
|Trans & Cap Improvements |97841 |
|Customer Service         |2889  |
|Metro Health             |5313  |
|Parks and Recreation     |19964 |
|Code Enforcement Services|321984|
|City Council             |34    |
+-------------------------+------+



In [32]:
# How do number of days late depend on department and request type?
(
    df.filter("case_closed")
#     .filter("case_late")
    .groupby("standardized_dept_name", "service_request_type")
    .agg(avg("num_days_late").alias("days_late"), count("*").alias("n_cases"))
    .withColumn("days_late", round(col("days_late"), 1))
    .sort(asc("days_late"))
    .show(40, truncate=False)
)

+------------------------+--------------------------------------------+---------+-------+
|standardized_dept_name  |service_request_type                        |days_late|n_cases|
+------------------------+--------------------------------------------+---------+-------+
|City Council            |Request for Research/Information            |null     |5      |
|Trans & Cap Improvements|Engineering Design                          |-1413.8  |1      |
|Trans & Cap Improvements|Signal Timing Modification By Engineer      |-1352.0  |22     |
|Animal Care Services    |Stray Animal                                |-998.8   |27346  |
|Parks & Recreation      |Major Park Improvement Install              |-278.3   |271    |
|Trans & Cap Improvements|Sidewalk Cost Sharing Program               |-177.8   |131    |
|DSD/Code Enforcement    |Multi Tenant Exterior                       |-135.7   |84     |
|DSD/Code Enforcement    |CPS Energy Towers                           |-129.8   |511    |
|DSD/Code 

In [36]:
(
    df.filter(col("service_request_type") == "Stray Animal")
    .select("case_opened_date", "case_due_date", "case_closed_date")
    .withColumn("days_to_resolve", datediff("case_due_date", "case_opened_date"))
    .show()
)

+-------------------+-------------------+-------------------+---------------+
|   case_opened_date|      case_due_date|   case_closed_date|days_to_resolve|
+-------------------+-------------------+-------------------+---------------+
|2018-01-01 00:42:00|2020-09-26 00:42:00|2018-01-01 12:29:00|            999|
|2018-01-01 08:39:00|2020-09-26 08:39:00|2018-01-01 12:30:00|            999|
|2018-01-01 09:51:00|2020-09-26 09:51:00|2018-01-01 11:53:00|            999|
|2018-01-01 10:39:00|2020-09-26 10:39:00|2018-01-01 12:29:00|            999|
|2018-01-01 10:44:00|2020-09-26 10:44:00|2018-01-01 12:43:00|            999|
|2018-01-01 10:52:00|2020-09-26 10:52:00|2018-01-01 11:53:00|            999|
|2018-01-01 11:11:00|2020-09-26 11:11:00|2018-01-01 11:53:00|            999|
|2018-01-01 11:46:00|2020-09-26 11:46:00|2018-01-01 12:26:00|            999|
|2018-01-01 11:58:00|2020-09-26 11:58:00|2018-01-01 12:26:00|            999|
|2018-01-01 12:28:00|2020-09-26 12:28:00|2018-01-01 12:44:00|   

## Handling Dates

Quick Recap: getting data from spark dataframes:

- `.show(n)`: prints the first `n` rows. Doesn't produce a value that can be used later
- `.first`: gives us the first row object
- `.head(n)`: gives us a list of the first `n` row objects
- `.collect()`: turns *all* the rows into a list of row objects **be careful here**

In [39]:
df.select(max('case_opened_date'), max("case_closed_date")).collect()

[Row(max(case_opened_date)=datetime.datetime(2018, 8, 8, 10, 38), max(case_closed_date)=datetime.datetime(2018, 8, 8, 10, 38))]

In [38]:
max_date = df.select(max('case_opened_date'), max('case_closed_date')).first()[0]
max_date

datetime.datetime(2018, 8, 8, 10, 38)

In [40]:
max_date = max_date.strftime('%Y-%m-%d %H:%M:%S')
max_date

'2018-08-08 10:38:00'

In [ ]:
df = (
    df.withColumn('case_age', datediff(lit(max_date), 'case_opened_date'))
    .withColumn('days_to_closed', datediff('case_closed_date', 'case_opened_date'))
    .withColumn('case_lifetime', when(col('case_closed'), col('days_to_closed')).otherwise(col('case_age')))
    .drop('case_age', 'days_to_closed')
)

## Sidebar: Python Code Formatting

Indentation conveys nesting

autoformatting tools remove the discussion

In [ ]:
df = (df.withColumn('case_age', datediff(lit(max_date), 'case_opened_date')).withColumn('days_to_closed', datediff('case_closed_date', 'case_opened_date')).withColumn('case_lifetime', when(col('case_closed'), col('days_to_closed')).otherwise(col('case_age'))).drop('case_age', 'days_to_closed'))

In [ ]:
df = (
    df.withColumn("case_age", datediff(lit(max_date), "case_opened_date"))
    .withColumn("days_to_closed", datediff("case_closed_date", "case_opened_date"))
    .withColumn(
        "case_lifetime",
        when(col("case_closed"), col("days_to_closed")).otherwise(col("case_age")),
    )
    .drop("case_age", "days_to_closed")
)

In [ ]:
(df.withColumn('case_age', datediff(lit(max_date), 'case_opened_date'))
 .withColumn('days_to_closed', datediff('case_closed_date', 'case_opened_date'))
 .withColumn('case_lifetime', when(col('case_closed'), col('days_to_closed')).otherwise(col('case_age')))
 .drop('case_age', 'days_to_closed'))

1. Install black -- the python code formatter

    ```
    python -m pip install black
    ```
    
1. Create an alias

    ```
    alias fmt-clipboard='pbpaste | black -q - | pbcopy'
    ```

    Put this line in your `.zshrc` or `.bash_profile`
    
    ```
    echo $0
    ```
    
    Will tell you what shell you are using
    
1. Open a new terminal

1. Copy the code you want to format

1. Run the `fmt-clipboard` command

1. Paste the formatted code


To format a `.py` file

```
black script.py
```

`black-nb`